How to authenticate to Google APIs?



In [3]:
// Load client secrets from a local file.var path = require('path');var fs = require('fs');var google = require('googleapis');var googleAuth = require('google-auth-library');// If modifying these scopes, delete your previously saved credentials// at ~/.credentials/calendar-nodejs-quickstart.jsonvar SCOPES = ['https://www.googleapis.com/auth/calendar.readonly'];var TOKEN_DIR = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, '.credentials');var SECRET_PATH = path.join(TOKEN_DIR, 'client_secret.json');var credentials = JSON.parse(fs.readFileSync(SECRET_PATH));

'/Users/briancullinan/jupytangular2/node_modules/google-auth-library/lib/auth/googleauth.js'

How to exchange token for oauth client?



In [4]:
/** * Store token to disk be used in later program executions. * * @param {Object} token The token to store to disk. */function storeToken(token, tokenPath) {    try {        fs.mkdirSync(TOKEN_DIR);    } catch (err) {        if (err.code != 'EEXIST') {            throw err;        }    }    fs.writeFileSync(tokenPath, JSON.stringify(token));    console.log('Token stored to ' + tokenPath);}/** * Create an OAuth2 client with the given credentials, and then execute the * given callback function. * * @param {Object} credentials The authorization client credentials. * @param {function} callback The callback to call with the authorized client. */var runSeleniumCell;var authorize = (scopes = SCOPES) => {    const tokenPath = path.join(TOKEN_DIR, scopes.join('')        .replace(/[^a-z]*/ig, '_') + '.json');    var clientSecret = credentials.installed.client_secret;    var clientId = credentials.installed.client_id;    var redirectUrl = credentials.installed.redirect_uris[0];    var auth = new googleAuth();    var oauth2Client = new auth.OAuth2(clientId, clientSecret, redirectUrl);    try {        // Check if we have previously stored a token.        var token = fs.readFileSync(tokenPath);        oauth2Client.credentials = JSON.parse(token);        return Promise.resolve(oauth2Client);    } catch (e) {
        if (e.code !== 'ENOENT') {            throw e;        } else {            var authUrl = oauth2Client.generateAuthUrl({                access_type: 'offline',                scope: scopes            });            return importer.import(['selenium cell'])                .then(runSeleniumCell => runSeleniumCell('authorize google access'))                .then(authorizeSelenium => authorizeSelenium(authUrl))                .then(code => new Promise((resolve, reject) => {                    oauth2Client.getToken(code, (err, token) => {                        if (err) {                            return reject(err);                        } else {                            return resolve(token);                        }                    });                }))                .then(token => {                    oauth2Client.credentials = token;                    storeToken(token, tokenPath);                    return oauth2Client;                })        }    }};module.exports = authorize;authorize;

[Function: authorize]

How to use selenium to authorize Google access?



In [6]:
var importer = require('../Core');var approveSelenium = (authUrl) => {    return client        .url(authUrl)        .loginGoogle()        .waitForVisible('#scope0 + button', 3000)        .moveToObject('#scope0')        .moveToObject('#submit_approve_access')        .waitForVisible('#submit_approve_access content', 3000)        .click('#submit_approve_access content')        .waitForVisible('#code', 3000)        .getValue('#code');};var client, getCredentials;var authorizeSelenium = (authUrl) => {    console.log('Authorizing: ' + authUrl);    return importer.import('selenium docker')        .then(seleniumDocker => importer.import('set up selenium server', {seleniumDocker}))        .then(getSeleniumServer => getSeleniumServer('act-selenium'))        .then(() => importer.import('webdriver client'))        .then(createClient => {            client = createClient('localhost', 4444);            return importer.import('decrypt password', client);        })        .then(r => {            getCredentials = r;            return importer.import('log in Google', {client, getCredentials});        })        .then(() => approveSelenium(authUrl))        .catch(e => console.log(e))};if (typeof client !== 'undefined'    && typeof client.authorizeSelenium === 'undefined') {    client.addCommand('authorizeSelenium', authorizeSelenium);}module.exports = authorizeSelenium;

ReferenceError: client is not defined